In [1]:
import pandas as pd 
import folium
import os
import geohash
from folium.plugins import HeatMap
from sklearn import datasets, linear_model
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error, mean_squared_log_error
from datetime import datetime
import numpy as np

In [2]:
RESULTS_DIR = "results"

In [3]:
df = pd.read_csv(os.path.join(RESULTS_DIR, "10_million_with_elevation_geohashes_timestamps.csv"))

In [4]:
df.head()

,Unnamed: 0,value,geohash,geohash5,timestamp
0,0,23.0,xn7tncn498k2,xn7tn,1.540076e+09
1,1,19.0,xnezj2x7rnh7,xnezj,1.540076e+09
2,2,15.0,xneysek46d7w,xneys,1.540076e+09
3,3,16.0,u336qgr9tvzw,u336q,1.540076e+09
4,4,18.0,xneyyr50hk99,xneyy,1.540076e+09


In [5]:
df = df[df.value >= 0]
df.head()

,Unnamed: 0,value,geohash,geohash5,timestamp
0,0,23.0,xn7tncn498k2,xn7tn,1.540076e+09
1,1,19.0,xnezj2x7rnh7,xnezj,1.540076e+09
2,2,15.0,xneysek46d7w,xneys,1.540076e+09
3,3,16.0,u336qgr9tvzw,u336q,1.540076e+09
4,4,18.0,xneyyr50hk99,xneyy,1.540076e+09


In [6]:
points_geohashes_prec_5 = df.geohash5.value_counts()
points_geohashes_prec_5.mean(), points_geohashes_prec_5.median()

(310.47663253261874, 34.0)

In [7]:
geohashes_for_regression = list(points_geohashes_prec_5.where(lambda count: count <= 1000).dropna().where(lambda count: count >= 50).dropna().index)
len(geohashes_for_regression)

10834

In [8]:
groupped_df = df[df.geohash5.isin(geohashes_for_regression)][["value", "geohash5", "timestamp"]].groupby("geohash5")
len(groupped_df)

10834

In [9]:
def prepare_svr_regresion_model(data):
    data = data.sort_values(by=['timestamp'])
    new_data = pd.DataFrame(data=[[]])
    size = data.shape[0]
    new_data['count'] = size
    
    if size <= 1:
        new_data['regresion'] = None
        new_data['reg_mean_squared_error'] = None
        new_data['reg_r2_score_error'] = None
        new_data['reg_median_absolute_error'] = None
        new_data['reg_mean_squared_log_error'] = None
        return new_data
    
    train_size = int(0.8 * size)
    test_size = size - train_size
    
    X_train = np.array(data.timestamp[:train_size]).reshape(-1, 1)
    X_test = np.array(data.timestamp[train_size:]).reshape(-1, 1)
    
    Y_train = np.ravel(np.array(data.value[:train_size]).reshape(-1, 1))
    Y_test = np.ravel(np.array(data.value[train_size:]).reshape(-1, 1))
    
    regr = SVR(kernel='sigmoid', C=1e3, gamma=0.1)

    regr.fit(X_train, Y_train)
    new_data['regresion'] = regr
    
    Y_pred = regr.predict(X_test)
    new_data['reg_mean_squared_error'] = count_function_of_none(mean_squared_error, Y_test, Y_pred)
    new_data['reg_r2_score_error'] = count_function_of_none(r2_score, Y_test, Y_pred)
    new_data['reg_median_absolute_error'] = count_function_of_none(median_absolute_error, Y_test, Y_pred)
    new_data['reg_mean_squared_log_error'] = count_function_of_none(mean_squared_log_error, Y_test, Y_pred)
    
    return new_data

def count_function_of_none(fun, Y_test, Y_pred):
    try:
        return fun(Y_test, Y_pred)
    except ValueError:
        return np.finfo(np.float64).max

In [10]:
df_after_training = groupped_df.apply(prepare_svr_regresion_model).reset_index()

In [11]:
df_after_training.head()

,geohash5,level_1,count,regresion,reg_mean_squared_error,reg_r2_score_error,reg_median_absolute_error,reg_mean_squared_log_error
0,6mc5m,0,167,"SVR(C=1000.0, cache_size=200, coef0=0.0, degre...",46.210000,-1.660827,4.4,0.052651
1,6mc5n,0,162,"SVR(C=1000.0, cache_size=200, coef0=0.0, degre...",45.545455,-1.039600,6.0,0.055709
2,6mc5p,0,70,"SVR(C=1000.0, cache_size=200, coef0=0.0, degre...",461.610000,-35.132412,22.4,0.225332
3,6mc5t,0,96,"SVR(C=1000.0, cache_size=200, coef0=0.0, degre...",19.750000,-0.890404,3.6,0.029383
4,6msem,0,119,"SVR(C=1000.0, cache_size=200, coef0=0.0, degre...",25.010000,-0.036759,3.6,0.008977


In [13]:
df_after_training_mean_squared = df_after_training[df_after_training.reg_mean_squared_error <= 100]
df_after_training_reg_r2_score = df_after_training[df_after_training.reg_r2_score_error <= 1000][df_after_training.reg_r2_score_error >= -5]
df_after_training_median_absolute = df_after_training[df_after_training.reg_median_absolute_error <= 10]
df_after_training_mean_log = df_after_training[df_after_training.reg_mean_squared_log_error <= 100]

In [14]:
def get_coordinates(geo):
    bbox = geohash.bbox(geo)
    return [[bbox["w"], bbox["n"]], [bbox["e"], bbox["n"]], [bbox["e"], bbox["s"]], [bbox["w"], bbox["s"]]]

def get_geo_data(ghash_df):
    features = []

    for geo, *_ in ghash_df.values:
        features.append({"type": "Feature", "id": geo, 
                         "geometry": {"type": "Polygon", "coordinates": [get_coordinates(geo)]}})

    return {"type": "FeatureCollection", "features": features}

def visualize_with_folium(df, error):
    m = folium.Map(location=[37.760806, 140.474722], zoom_start=10)

    m.choropleth(
        geo_data=get_geo_data(df),
        name='choropleth',
        data=df,
        columns=['geohash5', error],
        key_on='feature.id',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='error'
    )
    folium.LayerControl().add_to(m)
    return m

In [15]:
m_mean_squared = visualize_with_folium(df_after_training_mean_squared, 'reg_mean_squared_error')
m_reg_r2_score = visualize_with_folium(df_after_training_reg_r2_score, 'reg_r2_score_error')
m_median_absolute = visualize_with_folium(df_after_training_median_absolute, 'reg_median_absolute_error')
m_mean_log = visualize_with_folium(df_after_training_mean_log, 'reg_median_absolute_error')

In [16]:
m_mean_squared.save(os.path.join(RESULTS_DIR, "rbf_sigmoid_regression_mean_squared_errors.html"))
m_reg_r2_score.save(os.path.join(RESULTS_DIR, "rbf_sigmoid_regression_reg_r2_score_errors.html"))
m_median_absolute.save(os.path.join(RESULTS_DIR, "rbf_sigmoid_regression_median_absolute_errors.html"))
m_mean_log.save(os.path.join(RESULTS_DIR, "rbf_sigmoid_regression_mean_log_errors.html"))

In [21]:
single_regression = df_after_training[df_after_training.geohash5 == 'drt32']
single_regression

,geohash5,level_1,count,regresion,reg_mean_squared_error,reg_r2_score_error,reg_median_absolute_error,reg_mean_squared_log_error
1312,drt32,0,70,"SVR(C=1000.0, cache_size=200, coef0=0.0, degre...",24.752857,-10.662404,3.9,0.017157


In [59]:
single_geo_data = df[df.geohash5 == 'drt32'][['value', 'geohash5', 'timestamp']]
single_geo_data.head(n=10)

,value,geohash5,timestamp
6264464,33.0,drt32,1.524411e+09
6264465,33.0,drt32,1.524411e+09
6264466,36.0,drt32,1.524411e+09
6264467,37.0,drt32,1.524411e+09
6264468,36.0,drt32,1.524411e+09
6264469,35.0,drt32,1.524411e+09
6264470,36.0,drt32,1.524411e+09
6264471,40.0,drt32,1.524411e+09
6264472,40.0,drt32,1.524411e+09
6264473,37.0,drt32,1.524411e+09


In [50]:
single_regression = df_after_training[df_after_training.geohash5 == 'drt32'].iloc[0]['regresion']
type(single_regression)

sklearn.svm.classes.SVR

In [64]:
def get_timestamp(date):
    return datetime.strptime(date, '%Y-%m-%d %H:%M:%S').timestamp()

In [65]:
single_regression.predict([[get_timestamp('2019-01-15 00:00:00')]])

array([39.9])

In [63]:
single_geo_data.sort_values(by='timestamp').head()

,value,geohash5,timestamp
6264498,45.0,drt32,1.524411e+09
6267886,45.0,drt32,1.524411e+09
6267878,49.0,drt32,1.524411e+09
6264497,49.0,drt32,1.524411e+09
6267874,50.0,drt32,1.524411e+09


In [54]:
single_geo_data.timestamp.min(), single_geo_data.timestamp.max()

(1524410943.0, 1524411113.0)

In [32]:
def get_date(ts):
#     ts = int("1284101485")
    
    print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

In [61]:
get_date(single_geo_data.timestamp.min())

2018-04-22 15:29:03


In [62]:
get_date(single_geo_data.timestamp.max())

2018-04-22 15:31:53


In [57]:
single_geo_data[single_geo_data.timestamp == single_geo_data.timestamp.min()]

,value,geohash5,timestamp
6264498,45.0,drt32,1.524411e+09
6267886,45.0,drt32,1.524411e+09


In [66]:
single_geo_data[single_geo_data.timestamp == single_geo_data.timestamp.max()]

,value,geohash5,timestamp
6264464,33.0,drt32,1.524411e+09
6266725,33.0,drt32,1.524411e+09


In [67]:
def get_timestamp(date):
    return datetime.strptime(date, '%Y-%m-%d %H:%M:%S').timestamp()

In [68]:
single_regression.predict([[get_timestamp('2019-01-15 00:00:00')]])

array([39.9])